## Imports ##

In [2]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

In [41]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [42]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (20)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=32) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=32) # test -> 50% validation, 50% final test

In [43]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [44]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [45]:
batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model Training ##

In [46]:
import torch.nn as nn
import torch.optim as optim
from src.utils import OurModel


In [47]:
ml = OurModel()

In [48]:
# model configuration
config = nn.Sequential(
    nn.Linear(20, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.Tanh(),
    nn.Linear(32, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
)
ml.add_configuration(config)

In [55]:
#GPU optimalization TODO: Doesnt work
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [50]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [51]:
# Model Training
criterion = nn.BCEWithLogitsLoss()

epochs = 20
for epoch in range(epochs):
    ml.train()  # set model to training mode
    total_loss = 0
    correct = 0
    total = 0

    for inputs, true in train_loader:
        true = true.unsqueeze(1)
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.sigmoid(y_pred)              # convert logits → probabilities
        predicted = (preds > 0.5).float()          # threshold to 0/1
        correct += (predicted == true).sum().item()
        total += true.size(0)
    # Epoch summary
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")


Epoch [1/20] | Loss: 0.6702 | Accuracy: 0.5892
Epoch [2/20] | Loss: 0.6474 | Accuracy: 0.6209
Epoch [3/20] | Loss: 0.6433 | Accuracy: 0.6252
Epoch [4/20] | Loss: 0.6415 | Accuracy: 0.6270
Epoch [5/20] | Loss: 0.6402 | Accuracy: 0.6280
Epoch [6/20] | Loss: 0.6397 | Accuracy: 0.6274
Epoch [7/20] | Loss: 0.6386 | Accuracy: 0.6290
Epoch [8/20] | Loss: 0.6378 | Accuracy: 0.6309
Epoch [9/20] | Loss: 0.6379 | Accuracy: 0.6304
Epoch [10/20] | Loss: 0.6369 | Accuracy: 0.6314
Epoch [11/20] | Loss: 0.6366 | Accuracy: 0.6315
Epoch [12/20] | Loss: 0.6366 | Accuracy: 0.6307
Epoch [13/20] | Loss: 0.6360 | Accuracy: 0.6311
Epoch [14/20] | Loss: 0.6359 | Accuracy: 0.6326
Epoch [15/20] | Loss: 0.6355 | Accuracy: 0.6323
Epoch [16/20] | Loss: 0.6353 | Accuracy: 0.6328
Epoch [17/20] | Loss: 0.6350 | Accuracy: 0.6336
Epoch [18/20] | Loss: 0.6349 | Accuracy: 0.6337
Epoch [19/20] | Loss: 0.6350 | Accuracy: 0.6330
Epoch [20/20] | Loss: 0.6342 | Accuracy: 0.6333


In [52]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(ml.parameters(), lr=1e-4)

for epoch in range(20):
    total_loss = 0
    for inputs, true in train_loader:
        true = true.unsqueeze(1)
        inputs = inputs.float()
        true = true.float()
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        if torch.isnan(loss):
            print("NaN detected -> breaking")
            break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/20] | Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/20] | Loss: 0.6328
Epoch [2/20] | Loss: 0.6325
Epoch [3/20] | Loss: 0.6325
Epoch [4/20] | Loss: 0.6325
Epoch [5/20] | Loss: 0.6325
Epoch [6/20] | Loss: 0.6325
Epoch [7/20] | Loss: 0.6325
Epoch [8/20] | Loss: 0.6324
Epoch [9/20] | Loss: 0.6323
Epoch [10/20] | Loss: 0.6323
Epoch [11/20] | Loss: 0.6323
Epoch [12/20] | Loss: 0.6323
Epoch [13/20] | Loss: 0.6322
Epoch [14/20] | Loss: 0.6323
Epoch [15/20] | Loss: 0.6323
Epoch [16/20] | Loss: 0.6322
Epoch [17/20] | Loss: 0.6322
Epoch [18/20] | Loss: 0.6321
Epoch [19/20] | Loss: 0.6322
Epoch [20/20] | Loss: 0.6322


## Sweep ##

In [1]:
import wandb as wandb
from src.hyper_configs import sweep_config
wandb.login()
wandb.init(project="xbukovinam-faculty-of-informatics-and-infromation-techno", config=sweep_config)

wandb: Currently logged in as: xbukovinam (xbukovinam-faculty-of-informatics-and-infromation-techno) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [60]:
sweep_id = wandb.sweep(sweep=sweep_config, project="xbukovinam-faculty-of-informatics-and-infromation-techno")
CONFIG = wandb.config

SyntaxError: invalid syntax (1632951802.py, line 1)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=CONFIG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CONFIG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CONFIG.batch_size, shuffle=False)

In [ ]:
sw_nn = OurModel()

hl_sizes = []
if CONFIG.n_layers > 0:
    hl_sizes.append(CONFIG.hidden_size_1)
if CONFIG.n_layers > 1:
    hl_sizes.append(CONFIG.hidden_size_2)
if CONFIG.n_layers > 2:
    hl_sizes.append(CONFIG.hidden_size_2)

#Building the neural network from CONFIG
sw_nn = OurModel()
sw_nn.sweeping_build(20, 1, n_layers=CONFIG.n_layers, hidden_size=hl_sizes)
optimizer = getattr(torch.optim, CONFIG.optimizer)(sw_nn.parameters(), lr=CONFIG.learning_rate)
criterion = nn.BCEWithLogitsLoss()


for epoch in range(5):
        running_loss = 0.0
        for Xb, yb in train_loader:
            optimizer.zero_grad()
            preds = sw_nn(Xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        wandb.log({"epoch": epoch+1, "loss": avg_loss})

